In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import os
import tensorflow
tensorflow.keras.__version__
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

C:\Users\Diya\anaconda\envs\MachineLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Diya\anaconda\envs\MachineLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Diya\anaconda\envs\MachineLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Diya\anaconda\envs\MachineLearning\lib\s

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [4]:
X = df.drop("koi_disposition", axis=1)
y = df["koi_disposition"]
print(X.shape, y.shape)

(6991, 40) (6991,)


# Create a Train Test Split



In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [6]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data

X_scaler = MinMaxScaler().fit(X_train)



In [8]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
X_train_scaled.shape

(5243, 40)

# One-Hot Encoding


In [10]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [11]:
encoded_y_train.shape

(5243,)

In [12]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [13]:
y_train_categorical.shape

(5243, 3)

In [14]:
y_train_categorical

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

# Create the Deep Neural model



In [22]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=40))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=3, activation='sigmoid'))

In [23]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 200)               8200      
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 603       
Total params: 49,003
Trainable params: 49,003
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
5243/5243 - 0s - loss: 0.5409 - acc: 0.7210
Epoch 2/60
5243/5243 - 0s - loss: 0.3737 - acc: 0.8035
Epoch 3/60
5243/5243 - 0s - loss: 0.3642 - acc: 0.8133
Epoch 4/60
5243/5243 - 0s - loss: 0.3538 - acc: 0.8226
Epoch 5/60
5243/5243 - 0s - loss: 0.3448 - acc: 0.8293
Epoch 6/60
5243/5243 - 0s - loss: 0.3378 - acc: 0.8402
Epoch 7/60
5243/5243 - 0s - loss: 0.3324 - acc: 0.8417
Epoch 8/60
5243/5243 - 0s - loss: 0.3306 - acc: 0.8430
Epoch 9/60
5243/5243 - 0s - loss: 0.3229 - acc: 0.8470
Epoch 10/60
5243/5243 - 0s - loss: 0.3294 - acc: 0.8426
Epoch 11/60
5243/5243 - 0s - loss: 0.3135 - acc: 0.8550
Epoch 12/60
5243/5243 - 0s - loss: 0.3061 - acc: 0.8669
Epoch 13/60
5243/5243 - 0s - loss: 0.3083 - acc: 0.8629
Epoch 14/60
5243/5243 - 0s - loss: 0.3014 - acc: 0.8640
Epoch 15/60
5243/5243 - 0s - loss: 0.2955 - acc: 0.8684
Epoch 16/60
5243/5243 - 0s - loss: 0.2959 - acc: 0.8709
Epoch 17/60
5243/5243 - 0s - loss: 0.2994 - acc: 0.8686
Epoch 18/60
5243/5243 - 0s - loss: 0.2951 - acc: 0.8661
E

# Quantify our Trained Model



In [26]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: nan - acc: 0.9033
Normal Neural Network - Loss: nan, Accuracy: 0.9033181071281433


# Make Predictions


In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

# Save the Model

In [ ]:
# Save the model
model.save("DeepikaAwasthi_DeepNeuralmodel_trained.h5")